## Restaurant NER (AE - Aspect Extraction) By Distilbert

### Preprocess training data

In [1]:
import json
import pandas as pd

rest_ner_training_file = '/Users/sean.huang/Documents/my/books and courses/ztgg/Shared Code/BERT-for-ABSA/ae/rest/train.json'
rest_ner_training_samples = json.load(open(rest_ner_training_file))
len(rest_ner_training_samples)

1850

In [2]:
rest_ner_training_samples['0']

{'label': ['O', 'O', 'O', 'B'], 'sentence': ['I', 'LOVE', 'their', 'Thai']}

In [3]:
dict_rest_ner_training_samples = {'sentence_id':[], 'sentence':[], 'label':[]}

for key in rest_ner_training_samples.keys():
    dict_rest_ner_training_samples['sentence_id'].append(key)
    dict_rest_ner_training_samples['sentence'].append(rest_ner_training_samples[key]['sentence'])
    dict_rest_ner_training_samples['label'].append(rest_ner_training_samples[key]['label'])
    
dict_rest_ner_training_samples

df_rest_ner_training_samples = pd.DataFrame.from_dict(dict_rest_ner_training_samples)
df_rest_ner_training_samples

,sentence_id,sentence,label
0,0,"[I, LOVE, their, Thai]","[O, O, O, B]"
1,1,"[The, service, was, attentive, .]","[O, B, O, O, O]"
2,2,"[I, go, twice, a, month, !]","[O, O, O, O, O, O]"
3,3,"[The, food, was, average, to, above, -, averag...","[O, B, O, O, O, O, O, O, O, O, B, I, I, O, O, ..."
4,4,"[Ask, for, Usha, ,, the, nicest, bartender, in...","[O, O, B, O, O, O, O, O, O, O]"
...,...,...,...
1845,1845,"[You, can, get, a, table, without, a, reservat...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1846,1846,"[I, have, eaten, at, some, of, the, 'best, ', ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1847,1847,"[Worth, visiting, the, 1st, Ave, spot, because...","[O, O, O, B, I, I, O, O, O, O, O, O, O]"
1848,1848,"[Its, located, in, greenewich, village, .]","[O, O, O, O, O, O]"


In [5]:
df_rest_ner_training_samples_exploded = df_rest_ner_training_samples.explode(['sentence', 'label'])
df_rest_ner_training_samples_exploded

,sentence_id,sentence,label
0,0,I,O
0,0,LOVE,O
0,0,their,O
0,0,Thai,B
1,1,The,O
...,...,...,...
1849,1849,front,O
1849,1849,of,O
1849,1849,the,O
1849,1849,guest,O


In [6]:
def get_mapping(data, column):
    t2id = {}
    id2t = {}
 
    vocab = list(set(data[column].to_list()))
    
    id2t = {i:t for i, t in enumerate(vocab)}
    t2id = {t:i for  i, t in enumerate(vocab)}
    return t2id, id2t


token2id, id2token = get_mapping(df_rest_ner_training_samples_exploded, 'sentence')
tag2id, id2tag = get_mapping(df_rest_ner_training_samples_exploded, 'label')

print(list(token2id.items())[0:5], list(id2token.items())[0:5])
print(tag2id, id2tag)

[('Murray', 0), ('sure', 1), ('short', 2), ('watching', 3), ('outside', 4)] [(0, 'Murray'), (1, 'sure'), (2, 'short'), (3, 'watching'), (4, 'outside')]
{'I': 0, 'O': 1, 'B': 2} {0: 'I', 1: 'O', 2: 'B'}


In [7]:
df_rest_ner_training_samples_exploded['word_id'] = df_rest_ner_training_samples_exploded['sentence'].map(token2id)
df_rest_ner_training_samples_exploded['tag_id'] = df_rest_ner_training_samples_exploded['label'].map(tag2id)
df_rest_ner_training_samples_exploded.head(20)

,sentence_id,sentence,label,word_id,tag_id
0,0,I,O,468,1
0,0,LOVE,O,2687,1
0,0,their,O,22,1
0,0,Thai,B,2041,2
1,1,The,O,2709,1
1,1,service,B,2500,2
1,1,was,O,341,1
1,1,attentive,O,2372,1
1,1,.,O,2587,1
2,2,I,O,468,1


In [8]:
df_rest_ner_training_samples_exploded.isnull().sum()

sentence_id    0
sentence       0
label          0
word_id        0
tag_id         0
dtype: int64

In [9]:
df_rest_ner_training_samples = df_rest_ner_training_samples_exploded.groupby(['sentence_id'])['sentence', 'label', 'word_id', 'tag_id'].agg(lambda x: list(x))
df_rest_ner_training_samples

/var/folders/57/qcnnk6sn1vvbtwg1g1xc6kmw0000gp/T/ipykernel_92509/3037838837.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_rest_ner_training_samples = df_rest_ner_training_samples_exploded.groupby(['sentence_id'])['sentence', 'label', 'word_id', 'tag_id'].agg(lambda x: list(x))


,sentence,label,word_id,tag_id
sentence_id,,,,
0,"[I, LOVE, their, Thai]","[O, O, O, B]","[468, 2687, 22, 2041]","[1, 1, 1, 2]"
1,"[The, service, was, attentive, .]","[O, B, O, O, O]","[2709, 2500, 341, 2372, 2587]","[1, 2, 1, 1, 1]"
10,"[Everything, is, always, cooked, to, perfectio...","[O, O, O, O, O, O, O, O, B, O, O, O, O, B, O, ...","[3386, 810, 3419, 2207, 1483, 3748, 2449, 1646...","[1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, ..."
100,"[good, music, ,, great, food, ,, speedy, servi...","[O, B, O, O, B, O, O, B, O, O, O]","[1710, 2490, 2449, 306, 779, 2449, 3515, 2500,...","[1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1]"
1000,"[But, the, pizza, is, way, to, expensive, .]","[O, O, B, O, O, O, O, O]","[107, 1646, 3749, 810, 1148, 1483, 1478, 2587]","[1, 1, 2, 1, 1, 1, 1, 1]"
...,...,...,...,...
995,"[The, only, beverage, we, did, receive, was, w...","[O, O, O, O, O, O, O, O, O, O, O, O]","[2709, 1919, 1970, 555, 1721, 1588, 341, 3635,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
996,"[A, friend, from, Seattle, and, I, went, on, a...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[3487, 395, 480, 2256, 3407, 468, 1812, 1746, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
997,"[Went, on, a, 3, day, oyster, binge, ,, with, ...","[O, O, O, O, O, O, O, O, O, B, O, O, O, O, O, ...","[2893, 1746, 1800, 2269, 125, 3123, 2036, 2449...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, ..."


In [10]:
max_sent_length = max([len(each) for each in list(df_rest_ner_training_samples['sentence'])])
max_sent_length

74

### Load and finetune pretrained Bert model

In [11]:
from transformers import AutoModelForTokenClassification, AutoTokenizer

pretrained_bert_model_name = 'distilbert-base-uncased'
pretrained_bert_tokenizer = AutoTokenizer.from_pretrained(pretrained_bert_model_name)

In [12]:
tokens = pretrained_bert_tokenizer(list(df_rest_ner_training_samples['sentence'])[0], is_split_into_words=True)
tokens

{'input_ids': [101, 1045, 2293, 2037, 7273, 102], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [13]:
from datasets import Dataset

def align_ner_labels(samples):
    inputs = pretrained_bert_tokenizer(samples['sentence'], truncation=True, padding='max_length', max_length=128, is_split_into_words=True)

    label_input = []
    for i, label in enumerate(samples['label']):
        word_ids = inputs.word_ids(batch_index=i)
        
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            else:
                label_ids.append(tag2id[label[word_idx]])
        
        if i == 0:
            print(i)
            print(label)
            print(word_ids)
            print(label_ids)
            
        label_input.append(label_ids)
        
    inputs['labels'] = label_input
    
    # print('inputs', inputs.keys())
    
    return inputs

train_datasets = Dataset.from_pandas(df_rest_ner_training_samples[:1600]).map(align_ner_labels, batched=True, batch_size=100).remove_columns(['sentence', 'label', 'word_id', 'tag_id', 'sentence_id'])
validation_datasets = Dataset.from_pandas(df_rest_ner_training_samples[1600:]).map(align_ner_labels, batched=True, batch_size=10).remove_columns(['sentence', 'label', 'word_id', 'tag_id', 'sentence_id'])

Parameter 'function'=<function align_ner_labels at 0x12fb3eaf0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/16 [00:00<?, ?ba/s]

0
['O', 'O', 'O', 'B']
[None, 0, 1, 2, 3, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
[-100, 1, 1, 1, 2, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -10

  0%|          | 0/25 [00:00<?, ?ba/s]

0
['B', 'I', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
[None, 0, 0, 1, 2, 3, 4, 4, 5, 6, 7, 8, 9, 9, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
[-100, 2, 2, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -10

In [14]:
train_datasets

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1600
})

In [15]:
validation_datasets

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 250
})

In [16]:
import torch 

device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')   # mps is for Apple M1 GPU
device

device(type='mps')

In [17]:
from transformers import TrainingArguments, Trainer, DataCollatorForTokenClassification
import torch

pretrained_bert_model = AutoModelForTokenClassification.from_pretrained(pretrained_bert_model_name, num_labels=len(tag2id.keys()), id2label=id2tag, label2id=tag2id)

pretrained_bert_model.to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN t

DistilBertForTokenClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
          

In [18]:
args = TrainingArguments(
    output_dir='./rest_ner_model_distilbert',
    per_device_train_batch_size=10,
    per_device_eval_batch_size=2,
    num_train_epochs=10,
    logging_strategy='epoch',
    evaluation_strategy='epoch',
    report_to='none'
)

data_collator = DataCollatorForTokenClassification(pretrained_bert_tokenizer)
    
trainer = Trainer(
    model=pretrained_bert_model,
    args=args,
    train_dataset=train_datasets,
    eval_dataset=validation_datasets,
    # data_collator=data_collator, # for padding, optional here
    tokenizer=pretrained_bert_tokenizer
)

trainer.train()
trainer.save_model()


# solution to "AttributeError: module 'torch.distributed' has no attribute 'is_initialized'"
# https://stackoverflow.com/questions/72641886/attributeerror-module-torch-distributed-has-no-attribute-is-initialized-in/72641887

torch.distributed process group is initialized, but local_rank == -1. In order to use Torch DDP, launch your script with `python -m torch.distributed.launch
/Users/sean.huang/miniconda3/envs/m1_new/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1600
  Num Epochs = 10
  Instantaneous batch size per device = 10
  Total train batch size (w. parallel, distributed & accumulation) = 10
  Gradient Accumulation steps = 1
  Total optimization steps = 1600


Epoch,Training Loss,Validation Loss
1,0.195400,0.132977
2,0.072600,0.139341
3,0.031500,0.181322
4,0.014200,0.207930
5,0.005800,0.246490
6,0.002600,0.275901
7,0.001600,0.268061
8,0.000800,0.282526
9,0.000500,0.274736
10,0.000600,0.278625


***** Running Evaluation *****
  Num examples = 250
  Batch size = 2
***** Running Evaluation *****
  Num examples = 250
  Batch size = 2
***** Running Evaluation *****
  Num examples = 250
  Batch size = 2
Saving model checkpoint to ./rest_ner_model/checkpoint-500
Configuration saved in ./rest_ner_model/checkpoint-500/config.json
Model weights saved in ./rest_ner_model/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./rest_ner_model/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./rest_ner_model/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 250
  Batch size = 2
***** Running Evaluation *****
  Num examples = 250
  Batch size = 2
***** Running Evaluation *****
  Num examples = 250
  Batch size = 2
Saving model checkpoint to ./rest_ner_model/checkpoint-1000
Configuration saved in ./rest_ner_model/checkpoint-1000/config.json
Model weights saved in ./rest_ner_model/checkpoint-1000/pytorch_model.bin
tokenizer confi

In [20]:
from transformers import EarlyStoppingCallback


args = TrainingArguments(
    output_dir='./rest_ner_model_distilbert',
    per_device_train_batch_size=10,
    per_device_eval_batch_size=2,
    num_train_epochs=10,
    logging_strategy='steps',
    evaluation_strategy='steps',
    eval_steps = 100,
    load_best_model_at_end=True,
    report_to='none'
)

data_collator = DataCollatorForTokenClassification(pretrained_bert_tokenizer)
    
trainer = Trainer(
    model=pretrained_bert_model,
    args=args,
    train_dataset=train_datasets,
    eval_dataset=validation_datasets,
    # data_collator=data_collator, # for padding, optional here
    tokenizer=pretrained_bert_tokenizer,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()
trainer.save_model()


PyTorch: setting up devices
torch.distributed process group is initialized, but local_rank == -1. In order to use Torch DDP, launch your script with `python -m torch.distributed.launch
/Users/sean.huang/miniconda3/envs/m1_new/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1600
  Num Epochs = 10
  Instantaneous batch size per device = 10
  Total train batch size (w. parallel, distributed & accumulation) = 10
  Gradient Accumulation steps = 1
  Total optimization steps = 1600


Step,Training Loss,Validation Loss
100,No log,0.352193
200,No log,0.285597
300,No log,0.294611
400,No log,0.297299
500,0.008400,0.304166
600,0.008400,0.321328
700,0.008400,0.307595
800,0.008400,0.324975


***** Running Evaluation *****
  Num examples = 250
  Batch size = 2
***** Running Evaluation *****
  Num examples = 250
  Batch size = 2
***** Running Evaluation *****
  Num examples = 250
  Batch size = 2
***** Running Evaluation *****
  Num examples = 250
  Batch size = 2
***** Running Evaluation *****
  Num examples = 250
  Batch size = 2
Saving model checkpoint to ./rest_ner_model_distilbert/checkpoint-500
Configuration saved in ./rest_ner_model_distilbert/checkpoint-500/config.json
Model weights saved in ./rest_ner_model_distilbert/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./rest_ner_model_distilbert/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./rest_ner_model_distilbert/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 250
  Batch size = 2
***** Running Evaluation *****
  Num examples = 250
  Batch size = 2
***** Running Evaluation *****
  Num examples = 250
  Batch size = 2


Training completed. Do

### Load finetune Bert model for Rest NER task, and do prediction

In [21]:
from transformers import AutoModelForTokenClassification, AutoTokenizer

finetuned_rest_tokenizer = AutoTokenizer.from_pretrained('./rest_ner_model_distilbert')
finetuned_rest_model = AutoModelForTokenClassification.from_pretrained('./rest_ner_model_distilbert', num_labels=len(tag2id.keys()))

Didn't find file ./rest_ner_model_distilbert/added_tokens.json. We won't load it.
loading file ./rest_ner_model_distilbert/vocab.txt
loading file ./rest_ner_model_distilbert/tokenizer.json
loading file None
loading file ./rest_ner_model_distilbert/special_tokens_map.json
loading file ./rest_ner_model_distilbert/tokenizer_config.json
loading configuration file ./rest_ner_model_distilbert/config.json
Model config DistilBertConfig {
  "_name_or_path": "./rest_ner_model_distilbert",
  "activation": "gelu",
  "architectures": [
    "DistilBertForTokenClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "I",
    "1": "O",
    "2": "B"
  },
  "initializer_range": 0.02,
  "label2id": {
    "B": 2,
    "I": 0,
    "O": 1
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": fa

In [22]:
from transformers import pipeline

rest_ner_pipeline = pipeline('ner', model=finetuned_rest_model, tokenizer=finetuned_rest_tokenizer)

examples = ['Serves really good food, nice environment', 'Oh yeah ... the view was good too']
rest_ner_results = rest_ner_pipeline(examples)
print(rest_ner_results)

Disabling tokenizer parallelism, we're using DataLoader multithreading already


[[{'entity': 'B', 'score': 0.99975616, 'index': 4, 'word': 'food', 'start': 19, 'end': 23}, {'entity': 'B', 'score': 0.99974054, 'index': 7, 'word': 'environment', 'start': 30, 'end': 41}], [{'entity': 'B', 'score': 0.999783, 'index': 7, 'word': 'view', 'start': 16, 'end': 20}]]


### Evaluation

In [23]:
rest_ner_test_file = '/Users/sean.huang/Documents/my/books and courses/ztgg/Shared Code/BERT-for-ABSA/ae/rest/test.json'
rest_ner_test_samples = json.load(open(rest_ner_test_file))
len(rest_ner_test_samples)

676

In [24]:
dict_rest_ner_test_samples = {'sentence_id':[], 'sentence':[], 'full_sentence':[], 'label':[]}

for key in rest_ner_test_samples.keys():
    dict_rest_ner_test_samples['sentence_id'].append(key)
    dict_rest_ner_test_samples['sentence'].append(rest_ner_test_samples[key]['sentence'])
    dict_rest_ner_test_samples['full_sentence'].append(' '.join(rest_ner_test_samples[key]['sentence']))
    dict_rest_ner_test_samples['label'].append(rest_ner_test_samples[key]['label'])
    

In [25]:
rest_ner_results = rest_ner_pipeline(dict_rest_ner_test_samples['full_sentence'])

In [26]:
rest_ner_results[:5]

[[],
 [{'entity': 'B',
   'score': 0.9998142,
   'index': 4,
   'word': 'su',
   'start': 19,
   'end': 21},
  {'entity': 'B',
   'score': 0.99978334,
   'index': 5,
   'word': '##shi',
   'start': 21,
   'end': 24}],
 [{'entity': 'B',
   'score': 0.9990421,
   'index': 4,
   'word': 'portions',
   'start': 16,
   'end': 24}],
 [{'entity': 'B',
   'score': 0.9991598,
   'index': 1,
   'word': 'green',
   'start': 0,
   'end': 5},
  {'entity': 'I',
   'score': 0.9997466,
   'index': 2,
   'word': 'tea',
   'start': 6,
   'end': 9},
  {'entity': 'I',
   'score': 0.9999201,
   'index': 3,
   'word': 'cr',
   'start': 10,
   'end': 12},
  {'entity': 'I',
   'score': 0.9999298,
   'index': 4,
   'word': '##eme',
   'start': 12,
   'end': 15},
  {'entity': 'I',
   'score': 0.99991655,
   'index': 5,
   'word': 'br',
   'start': 16,
   'end': 18},
  {'entity': 'I',
   'score': 0.99991536,
   'index': 6,
   'word': '##ule',
   'start': 18,
   'end': 21},
  {'entity': 'I',
   'score': 0.9999141

In [27]:
reformatted_rest_ner_results = []

for sentence_result in rest_ner_results:
    if len(sentence_result) == 0:
        reformatted_rest_ner_results.append({})
        continue
    last_label = sentence_result[0]['entity']
    last_token = sentence_result[0]['word']
    reformatted_sent_result = {}
    for token_result in sentence_result[1:]:
        label = token_result['entity']
        token = token_result['word']
        
        if token.startswith('##') is False:
            if last_label != '':
                reformatted_sent_result[last_token] = last_label
            last_token = token
            last_label = label
        else:
            last_token = last_token + token[2:] # remove '##' 
            last_label = label            

    if last_label != '':
        reformatted_sent_result[last_token] = last_label
    
    reformatted_rest_ner_results.append(reformatted_sent_result)
    
reformatted_rest_ner_results[:5]         

[{},
 {'sushi': 'B'},
 {'portions': 'B'},
 {'green': 'B', 'tea': 'I', 'creme': 'I', 'brulee': 'I'},
 {'restaurant': 'B'}]

In [28]:
y_pred = []

for tokens, ner_tags in zip(dict_rest_ner_test_samples['sentence'], reformatted_rest_ner_results):
    
    pred = []
    for token in tokens:
        if token not in ner_tags.keys():
            pred.append('O')
            y_pred.append('O')
        else:
            pred.append(ner_tags[token])
            y_pred.append(ner_tags[token])
    # print(tokens, ner_tags, pred)
    
y_pred[:10]

['O', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'O']

In [29]:
len(y_pred)

10096

In [30]:
y_true = []
for each in dict_rest_ner_test_samples['label']:
    y_true.extend(each)

len(y_true)

10096

In [31]:
from sklearn import metrics
metrics.accuracy_score(y_true, y_pred)

0.9514659270998416

In [32]:
metrics.precision_recall_fscore_support(y_true, y_pred, labels=['B', 'I', 'O'])

(array([0.76714801, 0.66949153, 0.96958951]),
 array([0.69444444, 0.56428571, 0.98033464]),
 array([0.72898799, 0.6124031 , 0.97493247]),
 array([ 612,  280, 9204]))

In [33]:
metrics.confusion_matrix(y_true, y_pred, labels=['B', 'I', 'O'])

array([[ 425,    9,  178],
       [  17,  158,  105],
       [ 112,   69, 9023]])

## Home Work: do laptop NER (AE)

...

## Load Testing  - to do...

### Config File

### Result - Distil Bert

### Result - Bert

### Config File

### Result - Distil Bert

### Result - Bert